In [1]:
!python -V

Python 3.9.7


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
import mlflow

In [3]:
#os.environ["AWS_PROFILE"] = ""

TRACKING_SERVER_HOST = "ec2-3-79-25-97.eu-central-1.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-tilemachos/0', creation_time=1691323877605, experiment_id='0', last_update_time=1691323877605, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
data_both_cohorts = pd.read_csv('local_data/Debernardi_2020_data.csv')
data = data_both_cohorts[data_both_cohorts['patient_cohort'] == 'Cohort1'].copy() # Keep Cohort2 for later

In [4]:
data['diagnosis'] = data['diagnosis'].apply(lambda d: d-1)
dummies = pd.get_dummies(data['sex'], drop_first=True)
data = pd.concat([data.drop('sex', axis=1), dummies], axis=1)

In [7]:
categorical = ['M']
numerical = ['age','creatinine', 'LYVE1', 'REG1B', 'TFF1']
target = ['diagnosis']

In [8]:
X = data[categorical + numerical].values
y = np.squeeze(data[target].values) # Squeeze cause of dimension errors in XGBoost

In [9]:
# Perform the train - test - validation split here already
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

In [10]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [11]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dval, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(dval)
    print(f'y_val = {y_val}')

    print(f'y_pred = {y_pred}')

    #return {'loss': log_loss(y_val, y_pred), 'status': STATUS_OK}
    return {'loss': -roc_auc_score(y_val, y_pred, multi_class='ovr'), 'status': STATUS_OK}

In [12]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 15, 1)),
    'subsample': hp.uniform('subsample', .5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 7),
    'reg_alpha': hp.loguniform('reg_alpha', -10, 10),
    'reg_lambda': hp.loguniform('reg_lambda', -10, 10),
    'gamma': hp.loguniform('gamma', -10, 10),
    'objective': 'multi:softprob',
    #'eval_metric': 'mlogloss',
    'eval_metric': 'auc',
    'num_class': 3
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-auc:0.83300                            
[1]	validation-auc:0.84607                            
[2]	validation-auc:0.84236                            
[3]	validation-auc:0.84674                            
[4]	validation-auc:0.84467                            
[5]	validation-auc:0.83384                            
[6]	validation-auc:0.84075                            
[7]	validation-auc:0.83758                            
[8]	validation-auc:0.84210                            
[9]	validation-auc:0.84522                            
[10]	validation-auc:0.83802                           
[11]	validation-auc:0.83097                           
[12]	validation-auc:0.83487                           
[13]	validation-auc:0.84382                           
[14]	validation-auc:0.84371                           
[15]	validation-auc:0.83854                           
[16]	validation-auc:0.84394                           
[17]	validation-auc:0.83994                           
[18]	valid

In [36]:
best_result['max_depth'] = int(best_result['max_depth'])

In [35]:
best_model = xgb.train(
    params=best_result,
    dtrain=dtrain,
    num_boost_round=1000,
    )

In [37]:
y_pred = best_model.predict(xgb.DMatrix(X_test, label=y_test))
predictions = [round(value) for value in y_pred]

In [38]:
from sklearn.metrics import f1_score
test_score = f1_score(y_test, predictions, average='weighted')
acc_score = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions, average='weighted')
conf_matrix = confusion_matrix(y_test, predictions)